<a href="https://colab.research.google.com/github/hobbit0701/hobbit/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask,request,abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent,TextMessageContent
from linebot.v3.messaging import(
    Configuration,ApiClient,MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage,ConfirmTemplate,MessageAction
)

app = Flask(__name__)

@app.route('/',methods=['GET'])
def index():
  return 'Hello'

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route('/callback',methods=['POST'])
def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent,message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

    action = event.message.text
    if action == 'confirm':
       template = ConfirmTemplate(
           text='你喜歡中山大學嗎?',
           actions=[
               MessageAction(label='喜歡',text='快逃'),
               MessageAction(label='不喜歡',text='不喜自離')
           ]
       )
       reply = TemplateMessage(
           alt_text='confirm template',
           template=template
       )

    else:
      reply = TextMessage(text='Please type "confirm!"')
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[reply]
        )
    )


ngrok_start()
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://77c7-34-150-183-68.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:21:33] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:21:40] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:21:51] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:21:59] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:22:01] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:22:04] "POST /callback HTTP/1.1" 200 -
